# 동물분류

https://drive.google.com/file/d/1_d8RcCM21XneorFe_m4939erMkCnccS1/view?usp=drive_link

In [38]:
with open('../d_cnn/datasets/animals/translate.py') as f:
    content = f.readline().strip()
    # print(content)
    # 문자열 안에 있는 딕셔너리를 정상적으로 가져오기 위한 메소드 eval
    contents1 = eval(content[content.index("{"):content.index("}") + 1])
    contents2 = {v: k for k, v in contents1.items()}

print(contents1, contents2, sep='\n\n')


{'cane': 'dog', 'cavallo': 'horse', 'elefante': 'elephant', 'farfalla': 'butterfly', 'gallina': 'chicken', 'gatto': 'cat', 'mucca': 'cow', 'pecora': 'sheep', 'scoiattolo': 'squirrel', 'dog': 'cane', 'elephant': 'elefante', 'butterfly': 'farfalla', 'chicken': 'gallina', 'cat': 'gatto', 'cow': 'mucca', 'spider': 'ragno', 'squirrel': 'scoiattolo'}

{'dog': 'cane', 'horse': 'cavallo', 'elephant': 'elefante', 'butterfly': 'farfalla', 'chicken': 'gallina', 'cat': 'gatto', 'cow': 'mucca', 'sheep': 'pecora', 'squirrel': 'scoiattolo', 'cane': 'dog', 'elefante': 'elephant', 'farfalla': 'butterfly', 'gallina': 'chicken', 'gatto': 'cat', 'mucca': 'cow', 'ragno': 'spider', 'scoiattolo': 'squirrel'}


In [39]:
from glob import glob
import os

root = '../d_cnn/datasets/animals/original/'

directories = glob(os.path.join(root, '*'))
print(directories)

for directory in directories:
    old_name = os.path.basename(directory)  # 플랫폼 독립적으로 디렉토리 이름 추출
    try:
        new_name = contents1[old_name]
    except KeyError:
        new_name = contents2.get(old_name, old_name)  # old_name이 contents2에 없으면 그대로 유지
    new_directory = os.path.join(root, new_name)
    os.rename(directory, new_directory)
    

['../d_cnn/datasets/animals/original/gallina', '../d_cnn/datasets/animals/original/ragno', '../d_cnn/datasets/animals/original/gatto', '../d_cnn/datasets/animals/original/farfalla', '../d_cnn/datasets/animals/original/mucca', '../d_cnn/datasets/animals/original/cavallo', '../d_cnn/datasets/animals/original/cane', '../d_cnn/datasets/animals/original/pecora', '../d_cnn/datasets/animals/original/scoiattolo', '../d_cnn/datasets/animals/original/elefante']


In [40]:
root = '../d_cnn/datasets/animals/original/'
    
directories = glob(os.path.join(root, '*'))
directory_names = []
for directory in directories:
    directory_names.append(os.path.basename(directory))

print(directory_names)

['cat', 'butterfly', 'dog', 'sheep', 'spider', 'chicken', 'horse', 'squirrel', 'cow', 'elephant']


In [41]:
root = '../d_cnn/datasets/animals/original/'

for name in directory_names:
    for i, file_name in enumerate(os.listdir(os.path.join(root, name))):
        old_file = os.path.join(root + name + '/', file_name)
        new_file = os.path.join(root + name + '/', name + str(i + 1) + '.png')

        os.rename(old_file, new_file)

In [42]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_data_generator = ImageDataGenerator(rescale=1./255)

generator = image_data_generator.flow_from_directory(root, target_size=(150, 150), batch_size=32, class_mode='categorical')
print(generator.class_indices)

Found 26179 images belonging to 10 classes.
{'butterfly': 0, 'cat': 1, 'chicken': 2, 'cow': 3, 'dog': 4, 'elephant': 5, 'horse': 6, 'sheep': 7, 'spider': 8, 'squirrel': 9}


In [43]:
import pandas as pd

a_df = pd.DataFrame({'file_paths': generator.filepaths, 'targets': generator.classes})
a_df

,file_paths,targets
0,../d_cnn/datasets/animals/original/butterfly/b...,0
1,../d_cnn/datasets/animals/original/butterfly/b...,0
2,../d_cnn/datasets/animals/original/butterfly/b...,0
3,../d_cnn/datasets/animals/original/butterfly/b...,0
4,../d_cnn/datasets/animals/original/butterfly/b...,0
...,...,...
26174,../d_cnn/datasets/animals/original/squirrel/sq...,9
26175,../d_cnn/datasets/animals/original/squirrel/sq...,9
26176,../d_cnn/datasets/animals/original/squirrel/sq...,9
26177,../d_cnn/datasets/animals/original/squirrel/sq...,9


In [44]:
a_df.loc[:, 'file_paths'] = a_df.file_paths.apply(lambda x: x.replace('\\', '/'))
a_df

,file_paths,targets
0,../d_cnn/datasets/animals/original/butterfly/b...,0
1,../d_cnn/datasets/animals/original/butterfly/b...,0
2,../d_cnn/datasets/animals/original/butterfly/b...,0
3,../d_cnn/datasets/animals/original/butterfly/b...,0
4,../d_cnn/datasets/animals/original/butterfly/b...,0
...,...,...
26174,../d_cnn/datasets/animals/original/squirrel/sq...,9
26175,../d_cnn/datasets/animals/original/squirrel/sq...,9
26176,../d_cnn/datasets/animals/original/squirrel/sq...,9
26177,../d_cnn/datasets/animals/original/squirrel/sq...,9


In [45]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =\
train_test_split(a_df.file_paths, a_df.targets, stratify=a_df.targets, test_size=0.2, random_state=124)

print(y_train.value_counts())
print(y_test.value_counts())

targets
4    3890
8    3857
2    2478
6    2098
0    1690
3    1493
9    1490
7    1456
1    1334
5    1157
Name: count, dtype: int64
targets
4    973
8    964
2    620
6    525
0    422
3    373
9    372
7    364
1    334
5    289
Name: count, dtype: int64


In [47]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = \
train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=124)

print(y_train.value_counts())
print(y_val.value_counts())

targets
4    3112
8    3086
2    1982
6    1678
0    1352
3    1194
9    1192
7    1165
1    1067
5     926
Name: count, dtype: int64
targets
4    778
8    771
2    496
6    420
0    338
3    299
9    298
7    291
1    267
5    231
Name: count, dtype: int64


In [53]:
import shutil

root = '../d_cnn/datasets/animals/'

for file_path in X_train:
    # animal_dir을 경로 구분자로 분할하여 추출
    animal_dir = file_path[len(os.path.join(root, 'original')) + 1:file_path.rindex('/')]
    destination = os.path.join(root, 'train', animal_dir)

    # destination 디렉토리가 존재하지 않으면 생성
    if not os.path.exists(destination):
        os.makedirs(destination)

    # 파일을 destination 디렉토리로 복사
    shutil.copy2(file_path, destination)


In [54]:
import shutil

root = '../d_cnn/datasets/animals/'

for file_path in X_val:
    # animal_dir을 경로 구분자로 분할하여 추출
    animal_dir = file_path[len(os.path.join(root, 'original')) + 1:file_path.rindex('/')]
    destination = os.path.join(root, 'validation', animal_dir)

    # destination 디렉토리가 존재하지 않으면 생성
    if not os.path.exists(destination):
        os.makedirs(destination)

    # 파일을 destination 디렉토리로 복사
    shutil.copy2(file_path, destination)


In [55]:
import shutil

root = '../d_cnn/datasets/animals/'

for file_path in X_test:
    # animal_dir을 경로 구분자로 분할하여 추출
    animal_dir = file_path[len(os.path.join(root, 'original')) + 1:file_path.rindex('/')]
    destination = os.path.join(root, 'test', animal_dir)

    # destination 디렉토리가 존재하지 않으면 생성
    if not os.path.exists(destination):
        os.makedirs(destination)

    # 파일을 destination 디렉토리로 복사
    shutil.copy2(file_path, destination)
